# Task 2

## Tensorflow를 이용한 사용자 정의 모델

### 텐서와 연산

- 텐서(Tensor)는 다차원 배열로 Tensorflow의 기본 데이터 구조. Tensorflow 2.0 이후부터는 NumPy 배열과의 호환성이 기본으로 확보된다. GPU, TPU에서의 고성능 계산이 가능하다.
- 텐서 생성을 위해 `tf.constant()`를 사용하면, 연산은 자동으로 최적화되어 실행된다.

### 사용자 정의 손실 함수

- 손실 함수는 모델의 예측값과 실제값 사이 오차를 계산한다.


In [10]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.layers import Dense
import numpy as np

class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, units):
    super().__init__()
    self.units = units

  def build(self, input_shape):
    self.w = self.add_weight(name="kernel", shape=[input_shape[-1], self.units], initializer="glorot_uniform")
    self.b = self.add_weight(name="bias", shape=[self.units], initializer="zeros")

  def call(self, inputs):
    return tf.nn.relu(tf.matmul(inputs, self.w) + self.b)

def huber_loss(y_true, y_pred, delta=1.0):
  error_abs = tf.abs(y_true - y_pred)
  sl = 0.5 * (y - y_pred)**2
  ll = delta * (error_abs - 0.5 * delta)
  return tf.where(error_abs<= delta, sl, ll)

x, y = make_regression(random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
# 정수를 실수형으로, 0~1 범위로 정규화
x_train = x_train.astype(np.float32)/255.0
x_test = x_test.astype(np.float32)/255.0

model = Sequential([
    MyDenseLayer(32),
    MyDenseLayer(32),
    Dense(1, activation='linear', name='output'),
])
model.compile(
    optimizer=Adam(),
    loss=MeanSquaredError(),
    metrics=['accuracy']
)

model.fit(x_train, y_train, batch_size=32, epochs=10)
test_loss, test_acc = model.evaluate(x_test, y_test)
res = model.predict(x_test)
print(f"모델 훈련 중 손실 값 {test_loss:.4f}")
print("테스트 데이터에서의 MSE", MeanSquaredError(y_test, res))
print("테스트 샘플 1번 예측 값", res[0][0])
print("테스트 샘플 1번 실제 값", y_test[0])

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.0000e+00 - loss: 18466.1426
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0000e+00 - loss: 21998.4902 
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0000e+00 - loss: 18688.9473  
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0000e+00 - loss: 20964.0352 
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0000e+00 - loss: 18935.8633  
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 20418.7363 
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 20032.8145 
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 18775.5508 
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0000e+00 - loss: 20440.8730 
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 21550.7129 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.0000e+00 - loss: 17937.3145
1/1 ━━

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()